# Script for training a model

## Imports

In [1]:
import pandas as pd
import wandb
import numpy as np
from datasets import load_metric
from datasets import Dataset
from transformers import (AutoTokenizer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer)
from torch.utils.data import Dataset as D
import pickle

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Preparation for training

In [2]:
class MyDataset(D):
    def __init__(self, data_path):
        self.raw_data = pd.read_csv(data_path, sep='\t', index_col=0)
        
        data = pd.DataFrame()
        data['toxic'] = pd.concat([self.raw_data[self.raw_data['ref_tox'] > self.raw_data['trn_tox']]['reference'], self.raw_data[self.raw_data['ref_tox'] < self.raw_data['trn_tox']]['translation']])
        data['normal'] = pd.concat([self.raw_data[self.raw_data['ref_tox'] > self.raw_data['trn_tox']]['translation'], self.raw_data[self.raw_data['ref_tox'] < self.raw_data['trn_tox']]['reference']])
        data['toxic_reduction'] = abs(self.raw_data['ref_tox'] - self.raw_data['trn_tox'])
        self.data = data


    def __len__(self):
        return len(self.raw_data)

    def __getitem__(self, idx):
        return self.data.iloc[idx]

## Training

In [3]:
if __name__ == '__main__':
    
    # prepare the data
    df = pd.read_pickle("../../data/interim/text_dataset.pkl").data
    df = df[["toxic", "normal"]]
    dataset = Dataset.from_pandas(df).train_test_split(test_size=0.2)
    
    
    # tokenize text
    checkpoint = "t5-small"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    def preprocess_function(data):
        inputs = [l for l in data["toxic"]]
        targets = [l for l in data["normal"]]
        model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True, is_split_into_words=True)
        return model_inputs

    tokenized_dataset = dataset.map(preprocess_function, batched=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)
    
    
    # initialize trainer and start training
    folder = '../../models/T5-small' 

    training_args = Seq2SeqTrainingArguments(
        output_dir=folder,
        overwrite_output_dir=True,
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        save_steps=100,
        save_total_limit=3,
        num_train_epochs=10,
        predict_with_generate=True,
        fp16=True,
        generation_max_length = 133,
        weight_decay=1e-6,
        gradient_accumulation_steps=4,
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset['train'],
        eval_dataset=tokenized_dataset['test'],
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    
    wandb.login(key="secret")
    trainer.train()
    trainer.save_model('../../models/saved')
    
    # after you can choose either version from autosave, either version from saved folder

  0%|          | 0/463 [00:00<?, ?ba/s]

  0%|          | 0/116 [00:00<?, ?ba/s]

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dr-zhur0 (tttttttttt). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.9
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231104_210118-7wi6i9y9
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run mild-eon-1
wandb: ⭐️ View project at https://wandb.ai/tttttttttt/huggingface
wandb: 🚀 View run at https://wandb.ai/tttttttttt/huggingface/runs/7wi6i9y9
You're using a T5TokenizerFast tokenizer. Please note that with a fast tok

Epoch,Training Loss,Validation Loss
0,3.275200,2.996546
1,3.129900,2.907479
2,3.070900,2.863309
4,3.030300,2.833433
4,3.004100,2.814165
5,2.981100,2.802497
6,2.969600,2.789715
8,2.964900,2.783590
8,2.959400,2.780123
9,2.948200,2.779049


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u